# Importação de bibliotecas

In [81]:
from sentence_transformers import SentenceTransformer
from pycaret.classification import *
import pandas as pd
import joblib

# Criação do modelo

In [82]:
# Carregamento do dataset balanceado criado
df = pd.read_csv('phishing_dataset_CIS.csv')

# Junção do título e corpo do email
df['email_text'] = df['SubjectClear'] + ' ' + df['BodyClear']

In [83]:
# Gerar embeddings com um modelo eficiente
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['email_text'].tolist(), show_progress_bar=True)

# Transformar embeddings em DataFrame
embeddings_df = pd.DataFrame(embeddings)
embeddings_df.columns = [f'emb_{i}' for i in range(embeddings_df.shape[1])]

Batches:   0%|          | 0/866 [00:00<?, ?it/s]

In [84]:
# Adicionar variáveis auxiliares
X_aux = df[['time', 'urls', 'sendingPeriod']].copy()

# One-hot da variável categórica
X_aux = pd.get_dummies(X_aux, columns=['sendingPeriod'], drop_first=False).astype(int)

# Converter para lista
colunas_auxiliares = X_aux.columns.tolist()

# Salvar pra usar futuramente
joblib.dump(colunas_auxiliares, 'colunas_auxiliares.pkl')


['colunas_auxiliares.pkl']

In [85]:
# Concatenar embeddings com variáveis auxiliares
final_df = pd.concat([embeddings_df.reset_index(drop=True), X_aux.reset_index(drop=True)], axis=1)

# Adicional variável alvo phishing
final_df['phishing'] = df['phishing'].values

# Converter pra lista
colunas_treinamento = final_df.drop('phishing', axis=1).columns.tolist()

# Salvar pra usar futuramente
joblib.dump(colunas_treinamento, 'colunas_treinamento.pkl')

['colunas_treinamento.pkl']

In [86]:
clf = setup(
    data=final_df,
    target='phishing',
    session_id=123,
    train_size=0.8)

,Description,Value
0,Session id,123
1,Target,phishing
2,Target type,Binary
3,Original data shape,"(27701, 391)"
4,Transformed data shape,"(27701, 391)"
5,Transformed train set shape,"(22160, 391)"
6,Transformed test set shape,"(5541, 391)"
7,Numeric features,390
8,Preprocess,True
9,Imputation type,simple


In [87]:
# Obter o melhor modelo excluindo modelos que demoram muito para serem treinados e
# já mostraram desempenhos inferiores em testes passados

best_model = compare_models(exclude=['lightgbm', 'catboost', 'xgboost', 'gbc'])
evaluate_model(best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.9769,0.9963,0.9780,0.9769,0.9774,0.9539,0.9539,0.5260
ridge,Ridge Classifier,0.9755,0.9962,0.9774,0.9748,0.9761,0.9511,0.9511,0.2040
et,Extra Trees Classifier,0.9742,0.9974,0.9739,0.9756,0.9747,0.9484,0.9484,2.5540
lr,Logistic Regression,0.9735,0.9960,0.9757,0.9725,0.9741,0.9470,0.9470,1.1370
rf,Random Forest Classifier,0.9713,0.9965,0.9710,0.9728,0.9719,0.9426,0.9426,7.2930
svm,SVM - Linear Kernel,0.9639,0.9955,0.9754,0.9561,0.9652,0.9276,0.9288,0.4650
knn,K Neighbors Classifier,0.9528,0.9874,0.9789,0.9321,0.9549,0.9054,0.9066,0.8790
ada,Ada Boost Classifier,0.9313,0.9814,0.9351,0.9309,0.9329,0.8626,0.8627,10.4750
nb,Naive Bayes,0.9308,0.9831,0.9225,0.9408,0.9316,0.8615,0.8617,0.2080
qda,Quadratic Discriminant Analysis,0.8793,0.9602,0.9364,0.8946,0.9008,0.7559,0.7697,0.8470


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [88]:
# Salva o melhor modelo

save_model(best_model, 'modelo_phishing_final')
print("Modelo salvo em formato .pkl")

Transformation Pipeline and Model Successfully Saved
Modelo salvo em formato .pkl


# Criação da função para usar o modelo em emails externos

Pra usar o modelo em emails externos, as seguintes variáveis devem ser definidas:

In [89]:
modelo = load_model('modelo_phishing_final')
colunas_treinamento = joblib.load('colunas_treinamento.pkl')
colunas_auxiliares = joblib.load('colunas_auxiliares.pkl')
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

Transformation Pipeline and Model Successfully Loaded


Após definir as variáveis acima, o único parâmetro que precisa ser especificado é o email a ser testado, o qual deve seguir a seguinte formatação:

In [90]:
novo_email = {
    'SubjectClear': 'electrical circuits report available',
    'BodyClear': 'hello you have a pending report regarding electrical circuits that requires your immediate review access the document to verify important updates and confirm your information failure to respond may result in restrictions on your academic records please take action promptly',
    'time': 1,
    'urls': 0,
    'sendingPeriod': 'dawn'
}

A estrutura acima é a mesma utilizada no dataset de treino, e precisa manter essa formatação para que o modelo interprete os dados corretamente.

A forma como foram geradas e formatadas essas variáveis estão especificadas no documento ```00_DatasetGeneration.ipynb``` na parte de "Tratamento das Strings".

Com isso, a função para verificar o email é dada a seguir:

In [91]:
def isItPhishing(model=modelo,
                 training_columns = colunas_treinamento,
                 aux_columns=colunas_auxiliares,
                 embedding_model = embedding_model,
                 email_to_test = None):

    novo_df = pd.DataFrame([email_to_test])

    novo_df['email_text'] = novo_df['SubjectClear'] + ' ' + novo_df['BodyClear']

    embeddings = embedding_model.encode(novo_df['email_text'].tolist())
    embeddings_df = pd.DataFrame(embeddings, columns=[f'emb_{i}' for i in range(embeddings.shape[1])])

    X_aux = novo_df[['time', 'urls', 'sendingPeriod']].copy()
    X_aux = pd.get_dummies(X_aux, columns=['sendingPeriod'], drop_first=False).astype(int)

    # Alinhar variáveis auxiliares conforme treino
    for col in aux_columns:
        if col not in X_aux.columns:
            X_aux[col] = 0

    # Ordenar na mesma ordem do treino
    X_aux = X_aux[aux_columns]  

    input_final = pd.concat([embeddings_df.reset_index(drop=True), X_aux.reset_index(drop=True)], axis=1)

    for col in training_columns:
        if col not in input_final.columns:
            input_final[col] = 0

    # Ordena na mesma ordem do treino
    input_final = input_final[training_columns] 

    resultado = predict_model(model, data=input_final)
    label = resultado['prediction_label'][0]
    label_txt = 'phishing' if label == 1 else 'limpo'
    score = resultado['prediction_score'][0]*100
 
    return f"Resultado: o modelo tem {score:.2f}% de certeza de que o email recebido é {label_txt}."


Podemos testar com o email criado anteriormente:

In [92]:
isItPhishing(email_to_test=novo_email)

'Resultado: o modelo tem 87.75% de certeza de que o email recebido é phishing.'